In [1]:
%pip install torchvision
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, transforms, models
from torch.optim import lr_scheduler
import time
import copy
import os

# 1. Підготовка даних
data_dir = '.'  # Замість цього шляху вкажіть шлях до вашого набору даних
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], 
                             [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], 
                             [0.229, 0.224, 0.225])
    ]),
}

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), 
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], 
                                              batch_size=32, shuffle=True, num_workers=4)
               for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 2. Завантаження попередньо навченого ResNet
model = models.resnet50(pretrained=True)

# 3. Заморожування шарів
for param in model.parameters():
    param.requires_grad = False

# 4. Заміна вихідного шару
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(class_names))

model = model.to(device)

# 5. Визначення критерію та оптимізатора
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.fc.parameters(), lr=0.001)

# 6. Навчання моделі
num_epochs = 25
best_model_wts = copy.deepcopy(model.state_dict())
best_acc = 0.0
since = time.time()

for epoch in range(num_epochs):
    print(f'Epoch {epoch+1}/{num_epochs}')
    print('-' * 10)
    
    # Кожен епох розділений на тренувальний та валідаційний
    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()  # режим навчання
        else:
            model.eval()   # режим валідації
        
        running_loss = 0.0
        running_corrects = 0
        
        # Ітерація по даних
        for inputs, labels in dataloaders[phase]:
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            # Прямий прохід
            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
                
                # Зворотний прохід + оптимізація тільки в тренувальному режимі
                if phase == 'train':
                    loss.backward()
                    optimizer.step()
            
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
        
        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_acc = running_corrects.double() / dataset_sizes[phase]
        
        print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
        
        # Збереження найкращої моделі
        if phase == 'val' and epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_wts = copy.deepcopy(model.state_dict())
    
    print()
    
time_elapsed = time.time() - since
print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
print(f'Best val Acc: {best_acc:4f}')

# Завантаження найкращої моделі
model.load_state_dict(best_model_wts)

# Збереження моделі
torch.save(model.state_dict(), 'fine_tuned_resnet.pth')

Note: you may need to restart the kernel to use updated packages.


c:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/25
----------
train Loss: 0.8857 Acc: 0.4722
val Loss: 0.6321 Acc: 0.5500

Epoch 2/25
----------
train Loss: 0.6403 Acc: 0.6389
val Loss: 0.5014 Acc: 0.9500

Epoch 3/25
----------
train Loss: 0.4715 Acc: 0.9167
val Loss: 0.4064 Acc: 0.9000

Epoch 4/25
----------
train Loss: 0.3623 Acc: 0.9167
val Loss: 0.3308 Acc: 0.9500

Epoch 5/25
----------
train Loss: 0.2744 Acc: 0.9722
val Loss: 0.2686 Acc: 0.9500

Epoch 6/25
----------
train Loss: 0.2234 Acc: 0.9861
val Loss: 0.2250 Acc: 1.0000

Epoch 7/25
----------
train Loss: 0.1871 Acc: 0.9861
val Loss: 0.1898 Acc: 0.9500

Epoch 8/25
----------
train Loss: 0.1550 Acc: 0.9861
val Loss: 0.1761 Acc: 0.9500

Epoch 9/25
----------
train Loss: 0.1835 Acc: 0.9583
val Loss: 0.1786 Acc: 0.9500

Epoch 10/25
----------
train Loss: 0.1784 Acc: 0.9583
val Loss: 0.1481 Acc: 0.9500

Epoch 11/25
----------
train Loss: 0.1161 Acc: 0.9861
val Loss: 0.1203 Acc: 1.0000

Epoch 12/25
----------
train Loss: 0.0923 Acc: 1.0000
val Loss: 0.1237 Acc: 1.0000

E